In [1]:
#import streamlit as st
#import pandas as pd
#import altair as alt
#import plotly.express as px

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.ticker as mtick
import os
import platform
import glob

In [3]:
if platform.system()=='Windows': 
    #Directorios Windows
    os.chdir(
        r"G:\.shortcut-targets-by-id\120bUa35y3FJzvumWahpOx1qS9I6TIrhb\GIGANAV")

if platform.system()=='Darwin':
    os.chdir(
        '/Users/juliandiazparra/Library/CloudStorage/GoogleDrive-julian.adiaz@hotmail.com/Mi unidad/Negocios/Familiar/GIGANAV')

folder=os.getcwd()
data=os.path.join(folder,'Datos')
provisiones=os.path.join(data,'Provisiones')
facturas=os.path.join(data,'Facturas')
tickets=os.path.join(data,'Tickets')
usuarios=os.path.join(data,'Usuarios')
egresos=os.path.join(data,'Egresos')
transacciones=os.path.join(data,'Transacciones')
analisis=os.path.join(folder,'Operacion/Analisis')
resultados=os.path.join(analisis,'Resultados')
bases=os.path.join(analisis,'Bases_analisis')
outputs=os.path.join(analisis,'Outputs')
plots=os.path.join(outputs,'Gráficos')
usuarios_clean=os.path.join(bases,'Usuarios')
transac_clean=os.path.join(bases,'Transacciones')
tickets_clean=os.path.join(bases,'Tickets')
facturas_clean=os.path.join(bases,'Facturas')
egresos_clean=os.path.join(bases,'Egresos')
working=os.path.join(bases,'Bases_trabajo')
codigo=os.path.join(folder,'Operacion/Analisis/Código')
conciliaciones=os.path.join(folder,'Operacion/Operativo/Administrativo/Conciliaciones')

## Importación base de trabajo

In [4]:
list_of_files = glob.glob(os.path.join(outputs,'Base_tickets*.csv'))
latest_file = max(list_of_files, key=os.path.getctime)
working_data=pd.read_csv(latest_file, low_memory=False)
working_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15058 entries, 0 to 15057
Data columns (total 70 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15058 non-null  int64  
 1   id_ticket              15058 non-null  int64  
 2   departamento           15058 non-null  object 
 3   nombre                 15058 non-null  object 
 4   asunto                 15058 non-null  object 
 5   tecnico                15058 non-null  object 
 6   zona                   15021 non-null  object 
 7   abierto_por            15058 non-null  object 
 8   instal                 15058 non-null  int64  
 9   soporte                15058 non-null  int64  
 10  traslado               15058 non-null  int64  
 11  retiro                 15058 non-null  int64  
 12  mantenimiento          15058 non-null  int64  
 13  facturacion            15058 non-null  int64  
 14  cambio_plan            15058 non-null  int64  
 15  su

In [9]:
working_data.loc[working_data['tecnico_1'].notna(),'tecnico_1']

34       0.0
38       0.0
39       0.0
48       0.0
55       0.0
        ... 
4819     0.0
4910     0.0
4945     0.0
5040     0.0
11244    0.0
Name: tecnico_1, Length: 1609, dtype: float64

In [21]:
#Identifico ticket si cada tecnico aparece
working_data[['tecnico_1','tecnico_2']].fillna('', inplace=True)

#Creo variable de participacion de cada tecnico
for t in list(working_data.loc[
    working_data['tecnico_1'].notna(),
    'tecnico_1'].unique()):
    
    working_data.loc[
        :,t.split()[0]
    ]=np.select([(working_data['tecnico_1']==t) |
        (working_data['tecnico_2']==t)],[1],default=0)

/var/folders/_1/wccngqsd417dbsr4pcv82tj80000gn/T/ipykernel_2988/1201060034.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  working_data[['tecnico_1','tecnico_2']].fillna('', inplace=True)
/var/folders/_1/wccngqsd417dbsr4pcv82tj80000gn/T/ipykernel_2988/1201060034.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_data[['tecnico_1','tecnico_2']].fillna('', inplace=True)


AttributeError: 'numpy.float64' object has no attribute 'split'

In [ ]:
#Numero de tickets atendidos por tecnico/mes
num_tix_tecnicos=working_data.groupby(
    'mes_crd').agg(
    num_enc_Esneider=('Esneider','sum'), 
    num_enc_Alfonso=('Alfonso','sum'), 
    num_enc_Sair=('Sair','sum'), 
    num_enc_Leonardo=('Leonardo','sum'),
    num_enc_Jaime=('Jaime','sum')).reset_index().tail(8)#.iloc[:-1]
num_tix_tecnicos

In [ ]:
#Promedio tiempo de respuesta por tecnico/mes
temp_res_tecnicos_summary=working_data.loc[
    working_data['Esneider']==1
].groupby('mes_crd').agg(
    temp_res_esneider=('tiempo_resol',pd.Series.mean)).merge(
    working_data.loc[
    working_data['Alfonso']==1
].groupby('mes_crd').agg(
    temp_res_alfonso=('tiempo_resol',pd.Series.mean)), 
    left_index=True, right_index=True).merge(
    working_data.loc[
    working_data['Sair']==1
].groupby('mes_crd').agg(
    temp_res_sair=('tiempo_resol',pd.Series.mean)), 
    left_index=True, right_index=True).merge(
    working_data.loc[
    working_data['Leonardo']==1
].groupby('mes_crd').agg(
    temp_res_leonardo=('tiempo_resol',pd.Series.mean)), 
    left_index=True, right_index=True).reset_index()
temp_res_tecnicos_summary

In [ ]:
#Grafico tickets y tiempo respuesta promedio
fig,ax=plt.subplots()

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Esneider',
    kind='line',ax=ax,color='blue')

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Alfonso',
    kind='line',ax=ax,color='red')

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Sair',
    kind='line',ax=ax,color='black')

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Leonardo',
    kind='line',ax=ax,color='cyan')

ax.legend(['Esneider', 'Alfonso', 'Sair','Leonardo'])

plt.setp(ax, ylabel='Tickets', xlabel='')
plt.suptitle('Total tickets por técnico/mes\nGiganav Connections')
plt.tight_layout()
plt.show()

In [ ]:
#Grafico tickets y tiempo respuesta promedio
fig,ax=plt.subplots()

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Esneider',
    kind='line',ax=ax,color='blue')

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Alfonso',
    kind='line',ax=ax,color='red')

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Sair',
    kind='line',ax=ax,color='black')

num_tix_tecnicos.iloc[-4:].plot(
    x='mes_crd', y='num_enc_Leonardo',
    kind='line',ax=ax,color='cyan')

ax.legend(['Esneider', 'Alfonso', 'Sair','Leonardo'])

plt.setp(ax, ylabel='Tickets', xlabel='')
plt.suptitle('Total tickets por técnico/mes\nGiganav Connections')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax=plt.subplots()

temp_res_tecnicos_summary.iloc[-4:].plot(
    x='mes_crd', y='temp_res_esneider',
    kind='line',ax=ax,color='blue', grid=True)

temp_res_tecnicos_summary.iloc[-4:].plot(
    x='mes_crd', y='temp_res_alfonso',
    kind='line',ax=ax,color='red', grid=True)

temp_res_tecnicos_summary.iloc[-4:].plot(
    x='mes_crd', y='temp_res_sair',
    kind='line',ax=ax,color='black')

temp_res_tecnicos_summary.iloc[-4:].plot(
    x='mes_crd', y='temp_res_leonardo',
    kind='line',ax=ax,color='cyan')

ax.legend(['Esneider', 'Alfonso', 'Sair','Leonardo'])

plt.setp(ax, ylabel='Días', xlabel='')
plt.suptitle('Tiempo de resolución por técnico/mes\nGiganav Connections')
plt.tight_layout()
plt.show()

Comparo con asignación de técnicos en Mikrowisp vs encuesta de material

In [ ]:
#Miro cuantos tickets de los asignados en Mikrowisp
mkwp_summary_month=working_data.groupby(['mes_crd','tecnico']).agg(
    tickets_asignados=('id_ticket','nunique'), 
    temp_res=('tiempo_resolucion','mean'),
    num_enc_satisfaccion=('fecha_enc_sat','count'),
    promedio_satisfaccion=('calificacion_personal','mean'),
    Promotores=('promoter','sum')).reset_index().fillna(0)

empleados_names=[
    'ESNEIDER GARCIA BARAJAS','ALFONSO ARANGO TORO','SAIR BOLAÑO DURAN',
    'LEONARDO QUEZADA RAMIREZ','JAIME LEONARDO BARRERA BARAJAS',
    'MAYRA FERNANDA TRUJILLO SUAREZ','KAROLINA GUERRERO PRINCE',
    'EDGARD FONSECA','MIGUELPACHECO','No asignado']

mkwp_summary_month=mkwp_summary_month.loc[mkwp_summary_month['tecnico'].isin(empleados_names)]
mkwp_summary_month.loc[
    mkwp_summary_month['mes_crd']=='2024-08'].sort_values(
        'tickets_asignados',ascending=False)

In [ ]:
enc_tecnicos_mes_long=pd.melt(num_tix_tecnicos,id_vars='mes_crd')

enc_tecnicos_mes_long.loc[
    :,'variable']=enc_tecnicos_mes_long['variable'].str.lstrip('num_enc_')

enc_tecnicos_mes_long.rename(
    columns={'variable':'tecnico', 'value':'Encuestas respondidas'}, inplace=True)

enc_tecnicos_mes_long.loc[
    :,'tecnico'
]=np.select([enc_tecnicos_mes_long['tecnico'].str.contains('esneider',case=False),
             enc_tecnicos_mes_long['tecnico'].str.contains('alfonso',case=False),
             enc_tecnicos_mes_long['tecnico'].str.contains('sair',case=False),
             enc_tecnicos_mes_long['tecnico'].str.contains('leonardo',case=False),
             enc_tecnicos_mes_long['tecnico'].str.contains('jaime',case=False)],
            ['ESNEIDER GARCIA BARAJAS','ALFONSO ARANGO TORO','SAIR BOLAÑO DURAN',
             'LEONARDO QUEZADA RAMIREZ','JAIME LEONARDO BARRERA BARAJAS'],
            default='')

enc_tecnicos_mes_long.loc[
    enc_tecnicos_mes_long['mes_crd']=='2024-08']

In [ ]:
#Ahora pego datos mikrowisp con encuestas de material
summary_tecnico_mes=mkwp_summary_month.merge(
    enc_tecnicos_mes_long,
    on=['mes_crd','tecnico'],how='right').fillna(0)

summary_tecnico_mes.loc[
    :,'% encuesta material'
]=summary_tecnico_mes[
    'Encuestas respondidas']/summary_tecnico_mes[
    'tickets_asignados']

summary_tecnico_mes.loc[
    :,'% encuesta satisfaccion'
]=summary_tecnico_mes[
    'num_enc_satisfaccion']/summary_tecnico_mes[
    'tickets_asignados']

summary_tecnico_mes.loc[
    :,'Satisfaccion del usuario'
]=summary_tecnico_mes[
    'Promotores']/summary_tecnico_mes[
    'num_enc_satisfaccion']

summary_tecnico_mes.loc[
    summary_tecnico_mes['mes_crd']=='2024-08'
]

Numero de tickets por tecnico en el último mes

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-2]
].plot.bar(
    'tecnico','tickets_asignados',
    ax=ax, color='lightseagreen', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-2],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='Tickets asignados por ténico '+meses_estudio[-2],
        xlabel='')
plt.tight_layout()
plt.show()

In [ ]:
#Mes actual
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-1]
].plot.bar(
    'tecnico','tickets_asignados',
    ax=ax, color='lightseagreen', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-1],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='Tickets asignados por ténico '+meses_estudio[-1],
        xlabel='')
plt.tight_layout()
plt.show()

Tiempo de resolucion por tecnico en el último mes

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-2]
].plot.bar(
    'tecnico','temp_res',
    ax=ax, color='red', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-2],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='Tiempo de resolución por ténico '+meses_estudio[-2],
        xlabel='', ylabel='Días')

plt.tight_layout()
plt.show()

Mes actual

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-1]
].plot.bar(
    'tecnico','temp_res',
    ax=ax, color='red', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-1],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='Tiempo de resolución por ténico '+meses_estudio[-1],
        xlabel='', ylabel='Días')

plt.tight_layout()
plt.show()

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-2]
].plot.bar(
    'tecnico','% encuesta material',
    ax=ax, color='darkgreen', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-2],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='% encuestas material diligenciadas por ténico '+meses_estudio[-2],
        xlabel='', ylabel='')

import matplotlib.ticker as mtick
bars = ax.containers[0]
ax.bar_label(bars, labels = [f'{x.get_height():.0%}' for x in bars])
#ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.tight_layout()
plt.show()

Mes actual

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-1]
].plot.bar(
    'tecnico','% encuesta material',
    ax=ax, color='darkgreen', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-1],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='% encuestas material diligenciadas por ténico '+meses_estudio[-1],
        xlabel='', ylabel='')

import matplotlib.ticker as mtick
bars = ax.containers[0]
ax.bar_label(bars, labels = [f'{x.get_height():.0%}' for x in bars])
#ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.tight_layout()
plt.show()

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-2]
].plot.bar(
    'tecnico','num_enc_satisfaccion',
    ax=ax, color='blue', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-2],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='Num. encuestas satisfacción por ténico '+meses_estudio[-2],
        xlabel='', ylabel='')

import matplotlib.ticker as mtick
bars = ax.containers[0]
ax.bar_label(bars, labels = [f'{x.get_height()}' for x in bars])
#ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.tight_layout()
plt.show()

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

fig, ax=plt.subplots()

summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-2]
].plot.bar(
    'tecnico','promedio_satisfaccion',
    ax=ax, color='blue', 
    grid=True, legend=False)

ax.set_xticklabels([x for x in summary_tecnico_mes.loc[
                    summary_tecnico_mes['mes_crd']==meses_estudio[-2],
                    'tecnico']], rotation=45,size=7)

plt.setp(ax,title='% encuestas material diligenciadas por ténico '+meses_estudio[-2],
        xlabel='', ylabel='')

import matplotlib.ticker as mtick
bars = ax.containers[0]
ax.bar_label(bars, labels = [f'{x.get_height():.3}' for x in bars])
#ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.tight_layout()
plt.show()

Mes anterior

In [ ]:
#Satisfacción promedio por tecnico
summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-2],
    ['tecnico','num_enc_satisfaccion','promedio_satisfaccion']
]

Mes actual

In [ ]:
#Satisfacción promedio por tecnico
summary_tecnico_mes.loc[
    summary_tecnico_mes[
    'mes_crd']==meses_estudio[-1],
    ['tecnico','num_enc_satisfaccion','promedio_satisfaccion']
]

In [ ]:
meses_estudio=list(summary_tecnico_mes['mes_crd'].unique())[6:]

tecnicos_instaladores=list(summary_tecnico_mes.loc[
    summary_tecnico_mes['mes_crd']=='2024-08',
    'tecnico'
].unique())[:-1]

###ESCALAS DE DESEMPEÑO

resumen_tecnicos_inst={}
for m in meses_estudio:
    
    filepath=os.path.join(
    resultados,
    'Indicadores_tecnicos_instaladores_'+m+'.xlsx')
    
    writer = pd.ExcelWriter(filepath, 
                        engine = 'xlsxwriter') 
    
    resumen_tecnicos_inst[m]={}
    
    for t in tecnicos_instaladores:
        resumen_tecnicos_inst[m][t]=pd.DataFrame()
        resumen_tecnicos_inst[m][t].loc[
            :,t]=summary_tecnico_mes.loc[
            (summary_tecnico_mes['mes_crd']==m) & 
            (summary_tecnico_mes['tecnico']==t)].iloc[0].loc[[
                'mes_crd','tickets_asignados',
                'Encuestas respondidas','% encuesta material',
                'num_enc_satisfaccion','% encuesta satisfaccion',
                'Satisfaccion del usuario','promedio_satisfaccion','temp_res']]


        ###OJO META DE LOS TÉCNICOS MENSUAL
        resumen_tecnicos_inst[m][t].loc[
                :,'Meta']=[m,None,None,
                           
                           #PORCENTAJE ENCUESTAS MATERIAL RESPONDIDAS
                           .7,None,None,
                           #PROMOTORES
                           .7,None,
                           #TIEMPO RESOLUCIÓN
                           pd.to_timedelta(3.5, unit='d')]


        ##PORCENTAJE DE CUMPLIMIENTO
        resumen_tecnicos_inst[m][t].loc[:,
        '% Cumplimiento']=resumen_tecnicos_inst[m][t].loc[
        (resumen_tecnicos_inst[m][t].index=='% encuesta material') |
        (resumen_tecnicos_inst[m][t].index=='Satisfaccion del usuario')][m][t]/resumen_tecnicos_inst[m][t].loc[
        (resumen_tecnicos_inst[m][t].index=='% encuesta material') |
        (resumen_tecnicos_inst[m][t].index=='Satisfaccion del usuario')]['Meta']

        resumen_tecnicos_inst[m][t].loc[
                resumen_tecnicos_inst[m][t].index=='temp_res',
        '% Cumplimiento']=(1-(resumen_tecnicos_inst[m][t].loc[
                resumen_tecnicos_inst[m][t].index=='temp_res'][m][t]-
                              resumen_tecnicos_inst[m][t].loc[
                resumen_tecnicos_inst[m][t].index=='temp_res']['Meta'])/resumen_tecnicos_inst[m][t].loc[
                resumen_tecnicos_inst[m][t].index=='temp_res']['Meta'])

        ###CALIFICACION DEL DESEMPEÑO
        resumen_tecnicos_inst[m][t].loc[:,
        'Desempeño']=np.select([resumen_tecnicos_inst[m][t]['% Cumplimiento']>=1,
                                (resumen_tecnicos_inst[m][t]['% Cumplimiento']>=.8) &
                                (resumen_tecnicos_inst[m][t]['% Cumplimiento']<1), 
                                (resumen_tecnicos_inst[m][t]['% Cumplimiento']>=.5) &
                                (resumen_tecnicos_inst[m][t]['% Cumplimiento']<.8), 
                                (resumen_tecnicos_inst[m][t]['% Cumplimiento']>=0) &
                                (resumen_tecnicos_inst[m][t]['% Cumplimiento']<.5)],
                               ['Excelente','Destacado','Oportunidad','Crítico'], default='')
        
        resumen_tecnicos_inst[m][t].loc[
        'Promedio','% Cumplimiento']=resumen_tecnicos_inst[m][t][
        '% Cumplimiento'].mean()

        resumen_tecnicos_inst[m][t].loc[
        'Promedio','Desempeño']=np.select([resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']>=1,
                                (resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']>=.8) &
                                (resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']<1), 
                                (resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']>=.5) &
                                (resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']<.8), 
                                (resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']>=0) &
                                (resumen_tecnicos_inst[t].loc[
                                           'Promedio','% Cumplimiento']<.5)],
                               ['Excelente','Destacado','Oportunidad','Crítico'], default='')
        
        #resumen_tecnicos_inst[t].fillna(0, inplace=True)

        resumen_tecnicos_inst[t].index=['Mes','Tickets asignados','Encuestas material', '% encuesta material respondidas',
                                                       'Num. encuestas satisfaccion','% Enc. sat. respondidas', 'Satisfaccion del usuario',
                                                       'Promedio satisfacción','Tiempo resolución','Promedio']
        
        resumen_tecnicos_inst[t].to_excel(
                writer, sheet_name=t)

    writer.close()